<a href="https://colab.research.google.com/github/tonykipkemboi/Sentiment_Analysis_using_BERT_Neural_Network/blob/main/Kenyan_Embassy_BERT_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install & Import Dependencies

In [1]:
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1MB 1.5MB/s eta 0:13:36tcmalloc: large alloc 1147494400 bytes == 0x5596606da000 @  0x7f04f2989615 0x559626baacdc 0x559626c8a52a 0x559626badafd 0x559626c9efed 0x559626c21988 0x559626c1c4ae 0x559626baf3ea 0x559626c217f0 0x559626c1c4ae 0x559626baf3ea 0x559626c1e32a 0x559626c9fe36 0x559626c1d853 0x559626c9fe36 0x559626c1d853 0x559626c9fe36 0x559626c1d853 0x559626c9fe36 0x559626d223e1 0x559626c826a9 0x559626bedcc4 0x559626bae559 0x559626c224f8 0x559626baf30a 0x559626c1d3b5 0x559626c1c7ad 0x559626baf3ea 0x559626c1d3b5 0x559626baf30a 0x559626c1d3b5
     |████████████████▌               | 1055.7MB 1.3MB/s eta 0:12:28tcmalloc: large alloc 1434370048 bytes == 0x5596a4d30000 @  0x7f04f2989615 0x559626baacdc 0x559626c8a52a 0x559626badafd 0x559626c9efed 0x559626c21988 0x559626c1c4ae 0x559626baf3ea 0x559626c217f0 0x559626c1c4ae 0x559626baf3ea 0x559626c1e32a 0x559626c9fe36 0x559626c1d853 0

In [2]:
!pip3 install transformers requests beautifulsoup4 pandas numpy

     |████████████████████████████████| 2.5MB 8.2MB/s 
     |████████████████████████████████| 3.3MB 47.2MB/s 
     |████████████████████████████████| 901kB 44.5MB/s 


In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re

# Instantiate Model

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Encode & Calculate Sentiment

In [5]:
tokens = tokenizer.encode('love', return_tensors='pt')

In [6]:
result = model(tokens)

In [7]:
result.logits

tensor([[-1.4507, -1.5098, -0.3014,  0.8204,  1.9042]],
       grad_fn=<AddmmBackward>)

In [8]:
int(torch.argmax(result.logits))+1

5

# Collect Reviews

In [9]:
r = requests.get('https://www.yelp.com/biz/embassy-of-kenya-washington')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [10]:
reviews[0]

['I expected mediocre service and bad attitudes when I visited there but instead I was surprised by the professionalism of the staff there. They were very helpful and patient when dealing with customers and not at all what we are used to. I was shocked and amazed, and I want to congratulate them on a job well done.',
 "People were waiting before 10 am. They didn't allow us to view till 10:15 and only in their driveway and small side yard.Goods for sale and food. Ten dollars for a plate and I didn't like any of it. We were given a taste of tea and coffee. I got a cup of local lavender tea for a keurig machine. \xa0Disappointing.",
 "Calls and emails never get answered! Applied for Kenyan passport in January 2020 and this is August, no response yet! Even with Covid 19, it shouldn't take this long!! Some folks applied for ID's in January 2020 at this embassy in person, not yet received them. Were promised to be ready in 6 to 8 weeks, cargo planes are still flying from kenya to usa and vic

# Load Reviews into DataFrame & Score

In [13]:
df = pd.DataFrame(np.array(reviews), columns=['review'])
df.tail()

,review
3,Useless Useless Useless. Don't expect any help...
4,The embassy decided to join in the tour so lat...
5,New rule for Kenyans abroad is that you must h...
6,Calls will NEVER be returned if you have a que...
7,"Well, when my aunt visited Kenya last year, sh..."


In [14]:
df['review'].iloc[0]

'I expected mediocre service and bad attitudes when I visited there but instead I was surprised by the professionalism of the staff there. They were very helpful and patient when dealing with customers and not at all what we are used to. I was shocked and amazed, and I want to congratulate them on a job well done.'

In [15]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [16]:
sentiment_score(df['review'].iloc[1])

1

In [17]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [18]:
df

,review,sentiment
0,I expected mediocre service and bad attitudes ...,4
1,People were waiting before 10 am. They didn't ...,1
2,Calls and emails never get answered! Applied f...,1
3,Useless Useless Useless. Don't expect any help...,1
4,The embassy decided to join in the tour so lat...,2
5,New rule for Kenyans abroad is that you must h...,1
6,Calls will NEVER be returned if you have a que...,1
7,"Well, when my aunt visited Kenya last year, sh...",4


In [19]:
df['review'].iloc[0]

'I expected mediocre service and bad attitudes when I visited there but instead I was surprised by the professionalism of the staff there. They were very helpful and patient when dealing with customers and not at all what we are used to. I was shocked and amazed, and I want to congratulate them on a job well done.'